# Proprocessing dataset

In [1]:
with open('../dataset/train3_v4.csv') as f:
        print(f)

<_io.TextIOWrapper name='../dataset/train3_v4.csv' mode='r' encoding='cp950'>


In [2]:
import pandas as pd
import datasets
from datasets import load_dataset, Dataset, DatasetDict

Qdata_frame = pd.read_csv('../dataset/train3_v4.csv')
#Qdata_frame = pd.read_csv('../dataset/train3_v4.csv')


c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
Qdata_frame

,id,String,topic,type
0,1,Which of the following are components of an in...,Introduction to Information Processing,MC
1,2,Which of the following is an example of data c...,Introduction to Information Processing,MC
2,3,Which of the following personnels is responsib...,Introduction to Information Processing,MC
3,4,Which of the following is not a valid way to i...,Introduction to Information Processing,MC
4,5,Which of the following is/are (a) method(s) of...,Data Organisation and Data Control,MC
...,...,...,...,...
809,810,"In the case of (d)(i), will ALG1 or ALG2 be mo...","Algorithm Design, Program Development",LQ
810,811,What is the benefit of using an array to store...,Algorithm Design,LQ
811,812,There are different kinds of online shopping w...,"Database, Threats and Security on the Internet...",MC
812,813,A search engine can automatically the search r...,"Internet Services and Applications, Networking...",MC


In [4]:
duplicated = Qdata_frame.duplicated(subset=['String'])

In [5]:
duplicated

0      False
1      False
2      False
3      False
4      False
       ...  
809    False
810    False
811    False
812    False
813    False
Length: 814, dtype: bool

In [6]:
Qdata_frame = Qdata_frame.drop_duplicates(subset=['String'])

In [7]:
Qdata_frame['topic'] = Qdata_frame['topic'].apply(lambda x: x.strip().split(', '))

C:\Users\user\AppData\Local\Temp\ipykernel_27860\3268410748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Qdata_frame['topic'] = Qdata_frame['topic'].apply(lambda x: x.strip().split(', '))


In [8]:
Qdata_frame

,id,String,topic,type
0,1,Which of the following are components of an in...,[Introduction to Information Processing],MC
1,2,Which of the following is an example of data c...,[Introduction to Information Processing],MC
2,3,Which of the following personnels is responsib...,[Introduction to Information Processing],MC
3,4,Which of the following is not a valid way to i...,[Introduction to Information Processing],MC
4,5,Which of the following is/are (a) method(s) of...,[Data Organisation and Data Control],MC
...,...,...,...,...
809,810,"In the case of (d)(i), will ALG1 or ALG2 be mo...","[Algorithm Design, Program Development]",LQ
810,811,What is the benefit of using an array to store...,[Algorithm Design],LQ
811,812,There are different kinds of online shopping w...,"[Database, Threats and Security on the Interne...",MC
812,813,A search engine can automatically the search r...,"[Internet Services and Applications, Networkin...",MC


In [9]:
class_dict = {
'Algorithm Design': "Algorithm Design", 
'Communication Software and Protocols': "Networking and Internet Basics", 
'Computer System': "Computer System", 
'Data Manipulation and Analysis': "Data Manipulation and Analysis", 
'Data Organisation and Data Control': "Data Organisation and Data Control", 
'Database': "Spreadsheets and Databases", 
'Elementary Web Authoring': "Elementary Web Authoring", 
'Health and Ethical Issues': "Health and Ethical Issues", 
'Input and Output Devices': "Basic Machine Organisation", 
'Intellectual Property': "Intellectual Property", 
'Internet Services and Applications': "Internet Services and Applications", 
'Introduction to HTML': "Elementary Web Authoring", 
'Introduction to Information Processing': "Information Processing", 
'Multimedia Elements': "Multimedia Elements", 
'Number System': "Data Organisation and Data Control", 
'Problem- Formulation and Analysis': "Program Development", 
'Program Development': "Program Development", 
'Program Testing and Debugging': "Program Development", 
'Secondary Storage Devices': "Basic Machine Organisation", 
'Spreadsheet': "Spreadsheets and Databases", 
'System Software': "Computer System", 
'Networking and Internet Basics': 'Networking and Internet Basics',
'System Unit of a Computer System': "Basic Machine Organisation", 
'The Networking and Internet Basics': "Networking and Internet Basics", 
'Threats and Security on the Internet': "Threats and Security on the Internet", 
'Word Presentation': "Data Manipulation and Analysis"}

In [18]:
Qdata_frame['newTopic'] = Qdata_frame['topic'].apply(lambda topics: [class_dict[topic] for topic in topics])

C:\Users\user\AppData\Local\Temp\ipykernel_27860\1669245357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Qdata_frame['newTopic'] = Qdata_frame['topic'].apply(lambda topics: [class_dict[topic] for topic in topics])


In [19]:
Qdata_frame[:5]

,id,String,topic,type,newTopic
0,1,Which of the following are components of an in...,[Introduction to Information Processing],MC,[Information Processing]
1,2,Which of the following is an example of data c...,[Introduction to Information Processing],MC,[Information Processing]
2,3,Which of the following personnels is responsib...,[Introduction to Information Processing],MC,[Information Processing]
3,4,Which of the following is not a valid way to i...,[Introduction to Information Processing],MC,[Information Processing]
4,5,Which of the following is/are (a) method(s) of...,[Data Organisation and Data Control],MC,[Data Organisation and Data Control]


In [12]:
# labelling
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
Qdf = Qdata_frame.join(pd.DataFrame(mlb.fit_transform(Qdata_frame['newTopic']), columns=mlb.classes_))

In [13]:
# split the train, test, and validation data
import sklearn
from sklearn.model_selection import train_test_split

Qdf_train, Qdf_remain = train_test_split(Qdf, train_size=0.8)
Qdf_valid, Qdf_test = train_test_split(Qdf_remain, test_size=0.5)

In [14]:
Qdf_train.shape

(644, 20)

In [15]:
Qdf_test.shape

(81, 20)

In [16]:
Qdf_valid.shape

(81, 20)

In [ ]:
# change to hugging face dataset type
trainDS = Dataset.from_pandas(Qdf_train)
validDS = Dataset.from_pandas(Qdf_valid)
testDS = Dataset.from_pandas(Qdf_test)
ds = DatasetDict()
ds["train"] = trainDS
ds["valid"] = validDS
ds["test"] = testDS

labels_original = [label for label in ds['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [ ]:
Qdf[labels_original].drop(["id", "type", "topic", "newTopic"], axis=1).sum().sort_values()

In [ ]:
Qdf_train[labels_original].drop(["id", "type", "topic", "newTopic"], axis=1).sum().sort_values()

In [ ]:
Qdf_train[labels_original].drop(["id", "type", "topic", "newTopic"], axis=1).sum().sort_values().plot.bar()

In [ ]:
Qdf_valid[labels_original].drop(["id", "type", "topic", "newTopic"], axis=1).sum().sort_values().plot.bar()

In [ ]:
Qdf_valid[labels_original].drop(["id", "type", "topic", "newTopic"], axis=1).sum().sort_values()

In [ ]:
Qdf_test[labels_original].drop(["id", "type", "topic", "newTopic"], axis=1).sum().sort_values().plot.bar()

In [ ]:
Qdf_test[labels_original].drop(["id", "type", "topic", "newTopic"], axis=1).sum().sort_values()

In [ ]:
#  clear not used columns
columns = ds.column_names
columns_toRemove = ["id", "type", "topic", "newTopic"]
new_ds = ds.remove_columns(columns_toRemove)
print(new_ds)


In [ ]:
labels_original = [label for label in new_ds['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [ ]:
labels_original

In [ ]:
# extract the labels
labels_original
id2label = {idx:label for idx, label in enumerate(labels_original)}
label2id = {label:idx for idx, label in enumerate(labels_original)}

In [ ]:
import matplotlib.pyplot as plt
Qdf[labels_original].sum().plot.bar()

In [ ]:
#new_ds.save_to_disk('./encoded_data3_070')